In [ ]:
#----------------- Notebook for Hyperdrive & AutoML run ---------------------

In [21]:
##1. Initialise Workspace

In [22]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-134199")
exp = Experiment(workspace=ws, name="udacity-hyperdrive-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-134199
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-134199


In [23]:
##2. Create AmlCompute Cluster

In [44]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster1"

try: #--- Verify that cluster does not exist already
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
##3. Create Project Directory
##4. Create an estimator using SKLEARN
##5. Create hyperdrive config

In [26]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
#--from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "max_iter": choice(range(10,31)) ,
        "--C": uniform(0,10),
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./",
                    compute_target=cpu_cluster,
                    entry_script='train.py'
                   )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                  hyperparameter_sampling=ps, 
                  policy= policy,
                  primary_metric_name="Accuracy",
              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                  max_total_runs=20,
                  max_concurrent_runs=4)

In [27]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

In [28]:
import joblib
from azureml.core import Model
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

model= hyperdrive_best_run.register_model(model_name='udacity-hyperdrive-project', model_path='./outputs/model.pkl',model_framework=Model.Framework.SCIKITLEARN,model_framework_version='0.19.1')


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d265bf86-473c-4ea0-8d6e-0da4ff59fce4
Web View: https://ml.azure.com/experiments/udacity-hyperdrive-project/runs/HD_d265bf86-473c-4ea0-8d6e-0da4ff59fce4?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-134199/workspaces/quick-starts-ws-134199

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-10T11:14:14.451226][API][INFO]Experiment created<END>\n""<START>[2021-01-10T11:14:15.053090][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-10T11:14:15.6765165Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-10T11:14:15.568325][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d265bf86-473c-4ea0-8d6e-0da4ff59fce4
Web View: https://ml.azure.com/experiments/udacity-hyperdrive-project/runs/HD_d265bf86-473c-4ea0-8d6e-0da4ff59fce4?wsi

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


#split into train and test
X_train, X_text, Y_train, Y_test = train_test_split(x,y,test_size=0.30)

#join cleaned data
df = pd.concat([X_train,Y_train],axis=1)



In [49]:

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data= ds,
    label_column_name= 'y',
    compute_target= cpu_cluster,
    n_cross_validations=5)

In [50]:
# Submit your automl run

### YOUR CODE HERE ###
b_experiment = Experiment(ws, "automl_experiment")
automl_run = b_experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster1 with default configuration
Running on remote compute: cpu-cluster1
Parent Run ID: AutoML_9ea96181-1997-46ac-a830-ab9abf674090

Received interrupt. Returning now.

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == "Completed")

automl_best_run = automl_run.get_best_run_by_primary_metric()

model= automl_best_run.register_model(model_name='udacity-automl-project', model_path='./outputs/model.pkl',model_framework=Model.Framework.SCIKITLEARN,model_framework_version='0.19.1')
